In [2]:
!pip install sklearn-genetic
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.0 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import SGDRegressor, LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.feature_selection import SelectFromModel, SequentialFeatureSelector
from genetic_selection import GeneticSelectionCV
import optuna

In [4]:
# Load the dataset
from google.colab import drive # Load data
drive.mount('/content/drive')

train_data = pd.read_csv('/content/drive/MyDrive/Masters/ML/train_house.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Masters/ML/test_House.csv')

Mounted at /content/drive


In [5]:
train_data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [6]:
print(train_data.shape)
print(test_data.shape)

(1460, 81)
(1459, 80)


In [7]:
print(train_data.isnull().sum())

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64


In [8]:
# Assume target variable is "SalePrice" in train data
X_train_full = train_data.drop("SalePrice", axis=1)
y_train_full = train_data["SalePrice"]
X_test_full = test_data.copy()

In [9]:
# Identify numeric and categorical features
numeric_features = X_train_full.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train_full.select_dtypes(include=['object']).columns

# Preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [10]:
# Split training data into train/validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

In [11]:
# Define base Models
models = {
    "Random Forest": RandomForestRegressor(random_state=42),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "Support Vector Regressor": SVR(),
    "MLP Regressor": MLPRegressor(random_state=42, max_iter=1000),
    "SGD Regressor": SGDRegressor(random_state=42)
}

In [12]:
# Train and Evaluate Models
results = {}
for name, model in models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                ('regressor', model)])
    scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
    results[name] = -np.mean(scores)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perce

In [13]:
# Stacking Ensemble
class StackingModel(BaseEstimator, RegressorMixin):
    def __init__(self, models, meta_model):
        self.models = models
        self.meta_model = meta_model

    def fit(self, X, y):
        self.model_instances = []
        for name, model in self.models.items():
            pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                        ('regressor', model)])
            pipeline.fit(X, y)
            self.model_instances.append((name, pipeline))

        # Generate meta features
        meta_features = np.column_stack([model.predict(X) for _, model in self.model_instances])
        self.meta_model.fit(meta_features, y)
        return self

    def predict(self, X):
        meta_features = np.column_stack([model.predict(X) for _, model in self.model_instances])
        return self.meta_model.predict(meta_features)

stacked_model = StackingModel(models=models, meta_model=LinearRegression())
stacked_model.fit(X_train, y_train)
stacked_predictions = stacked_model.predict(X_val)
stacked_rmse = mean_squared_error(y_val, stacked_predictions, squared=False)
results["Stacked Model"] = stacked_rmse

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [29]:
 #Predict on Test Data
stacked_test_predictions = stacked_model.predict(X_test_full)

# Save Predictions
submission = pd.DataFrame({
    "Id": X_test_full["Id"],
    "SalePrice": stacked_test_predictions
})
submission.to_csv("submission_Stacked_Ujwala.csv", index=False)

In [20]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

# Optimized parameter distributions for RandomizedSearchCV
param_distributions = {
    "Random Forest": {
        "regressor__n_estimators": randint(50, 150),
        "regressor__max_depth": [10, 15, None]
    },
    "Decision Tree": {
        "regressor__max_depth": [10, 15, None],
        "regressor__min_samples_split": randint(2, 10)
    },
    "Gradient Boosting": {
        "regressor__n_estimators": randint(50, 150),
        "regressor__learning_rate": uniform(0.05, 0.15)
    },
    "Support Vector Regressor": {
        "regressor__C": uniform(0.1, 5),
        "regressor__epsilon": uniform(0.01, 0.1),
        "regressor__kernel": ["linear", "rbf"]
    },
    "MLP Regressor": {
        "regressor__hidden_layer_sizes": [(50,), (100,)],
        "regressor__alpha": uniform(0.0001, 0.01),
        "regressor__learning_rate": ["constant", "adaptive"]
    },
    "SGD Regressor": {
        "regressor__alpha": uniform(0.0001, 0.01),
        "regressor__penalty": ["l2", "elasticnet"],
        "regressor__max_iter": randint(1000, 2000)
    }
}

# Dictionary to store tuned models
tuned_models = {}

# Perform hyperparameter tuning for each model, predict on test data, and save results
for name, model in models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', model)])

    # Check if parameter distributions are defined for the model
    if name in param_distributions:
        print(f"Tuning hyperparameters for {name} using RandomizedSearchCV...")
        random_search = RandomizedSearchCV(
            pipeline,
            param_distributions=param_distributions[name],
            n_iter=20,
            cv=2,
            scoring='neg_root_mean_squared_error',
            n_jobs=-1,
            random_state=42
        )
        random_search.fit(X_train, y_train)
        best_model = random_search.best_estimator_
        print(f"Best parameters for {name}: {random_search.best_params_}")
        tuned_models[name] = best_model  # Save the tuned model
    else:
        print(f"No hyperparameter tuning defined for {name}, using default parameters.")
        pipeline.fit(X_train, y_train)
        tuned_models[name] = pipeline  # Save the default model

    # Predict on test data
    test_predictions = tuned_models[name].predict(X_test_full)

    # Save predictions to a CSV file
    submission = pd.DataFrame({
        "Id": X_test_full["Id"],
        "SalePrice": test_predictions
    })
    submission_file = f"submission_{name.replace(' ', '_')}.csv"
    submission.to_csv(submission_file, index=False)
    print(f"Predictions for {name} saved to {submission_file}")



Tuning hyperparameters for Random Forest using RandomizedSearchCV...
Best parameters for Random Forest: {'regressor__max_depth': 15, 'regressor__n_estimators': 113}
Predictions for Random Forest saved to submission_Random_Forest.csv
Tuning hyperparameters for Decision Tree using RandomizedSearchCV...
Best parameters for Decision Tree: {'regressor__max_depth': None, 'regressor__min_samples_split': 4}
Predictions for Decision Tree saved to submission_Decision_Tree.csv
Tuning hyperparameters for Gradient Boosting using RandomizedSearchCV...
Best parameters for Gradient Boosting: {'regressor__learning_rate': 0.17992642186624025, 'regressor__n_estimators': 149}
Predictions for Gradient Boosting saved to submission_Gradient_Boosting.csv
Tuning hyperparameters for Support Vector Regressor using RandomizedSearchCV...
Best parameters for Support Vector Regressor: {'regressor__C': 5.016154429033941, 'regressor__epsilon': 0.056676289324798, 'regressor__kernel': 'linear'}
Predictions for Support V

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Best parameters for MLP Regressor: {'regressor__alpha': 0.0003058449429580245, 'regressor__hidden_layer_sizes': (100,), 'regressor__learning_rate': 'adaptive'}
Predictions for MLP Regressor saved to submission_MLP_Regressor.csv
Tuning hyperparameters for SGD Regressor using RandomizedSearchCV...
Best parameters for SGD Regressor: {'regressor__alpha': 0.007419939418114052, 'regressor__max_iter': 1700, 'regressor__penalty': 'l2'}
Predictions for SGD Regressor saved to submission_SGD_Regressor.csv


In [21]:
# Filter tuned models to use only Gradient Boosting, Random Forest, and SGD Regressor
base_models_tuned = {
    "Gradient Boosting": tuned_models["Gradient Boosting"],
    "Random Forest": tuned_models["Random Forest"],
    "SGD Regressor": tuned_models["SGD Regressor"]
}

# Define and tune the meta-model using Optuna
class StackingModel(BaseEstimator, RegressorMixin):
    def __init__(self, models, meta_model):
        self.models = models
        self.meta_model = meta_model

    def fit(self, X, y):
        self.model_instances = []
        meta_features = []

        for name, model in self.models.items():
            model.fit(X, y)
            self.model_instances.append((name, model))
            meta_features.append(model.predict(X))

        self.meta_features_train = np.column_stack(meta_features)
        self.meta_model.fit(self.meta_features_train, y)
        return self

    def predict(self, X):
        meta_features = np.column_stack([
            model.predict(X) for _, model in self.model_instances
        ])
        return self.meta_model.predict(meta_features)

# Define Optuna objective function for meta-model tuning
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    learning_rate = trial.suggest_float('learning_rate', 0.05, 0.2)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)

    meta_model = GradientBoostingRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        random_state=42
    )

    # Build the stacking model using only the selected base models
    stacked_model = StackingModel(models=base_models_tuned, meta_model=meta_model)
    stacked_model.fit(X_train, y_train)

    # Validate the stacking model
    stacked_predictions = stacked_model.predict(X_val)
    stacked_rmse = mean_squared_error(y_val, stacked_predictions, squared=False)
    return stacked_rmse

# Perform Optuna optimization for the meta-model
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

# Best meta-model parameters
best_params = study.best_params
print(f"Best parameters for the meta-model: {best_params}")

# Final Stacking Model with the best meta-model
best_meta_model = GradientBoostingRegressor(
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    random_state=42
)
stacked_model = StackingModel(models=base_models_tuned, meta_model=best_meta_model)
stacked_model.fit(X_train, y_train)

# Validate the stacked model
stacked_predictions = stacked_model.predict(X_val)
stacked_rmse = mean_squared_error(y_val, stacked_predictions, squared=False)
print(f"Stacked Model RMSE: {stacked_rmse}")

# Predict on test data
stacked_test_predictions = stacked_model.predict(X_test_full)

# Save predictions
submission = pd.DataFrame({
    "Id": X_test_full["Id"],  # Replace 'Id' with the test set identifier column
    "SalePrice": stacked_test_predictions
})
submission.to_csv("submission_Stacked_Optimized.csv", index=False)
print("Predictions saved to submission_Stacked_Optimized.csv")


[I 2024-11-27 02:10:56,256] A new study created in memory with name: no-name-b1120939-9f87-4efe-93d8-c3348f8a550d
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-11-27 02:11:11,809] Trial 0 finished with value: 28250.625756376216 and parameters: {'n_estimators': 189, 'learning_rate': 0.13945919992062672, 'max_depth': 6, 'min_samples_split': 10}. Best is trial 0 with value: 28250.625756376216.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-11-27 02:11:30,772] Trial 1 finished with value: 29378.488525274497 and parameters: {'n_estimators': 109, 'learning_

Best parameters for the meta-model: {'n_estimators': 88, 'learning_rate': 0.10282696370290262, 'max_depth': 5, 'min_samples_split': 6}
Stacked Model RMSE: 27956.342447296985
Predictions saved to submission_Stacked_Optimized.csv


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [22]:
# Load 'train_100' dataset
train_100 = pd.read_csv('/content/drive/MyDrive/Masters/ML/train_house_100.csv')
X_100 = train_100.drop("SalePrice", axis=1)
y_100 = train_100["SalePrice"]

# Identify numeric and categorical features in train_100
numeric_features_100 = X_100.select_dtypes(include=['int64', 'float64']).columns
categorical_features_100 = X_100.select_dtypes(include=['object']).columns

# Coerce numeric features to ensure valid data (e.g., replace invalid entries with NaN)
for col in numeric_features_100:
    X_100[col] = pd.to_numeric(X_100[col], errors='coerce')

# Define preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Handle missing numeric values
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Handle missing categorical values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Define a column transformer for preprocessing
preprocessor_100 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features_100),
        ('cat', categorical_transformer, categorical_features_100)
    ])

# Fit and transform the training data
X_100_preprocessed = preprocessor_100.fit_transform(X_100)

# Inspect transformed data
print(f"Preprocessed training data shape: {X_100_preprocessed.shape}")
print(f"Numeric features: {numeric_features_100}")
print(f"Categorical features: {categorical_features_100}")


Preprocessed training data shape: (99, 217)
Numeric features: Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'PoolQC', 'MiscVal', 'MoSold', 'YrSold'],
      dtype='object')
Categorical features: Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual'

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['PoolQC']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


In [27]:
# Feature selection using SelectFromModel
selector_model = RandomForestRegressor(random_state=42)
select_from_model = SelectFromModel(selector_model).fit(X_100_preprocessed, y_100)
X_100_selected = select_from_model.transform(X_100_preprocessed)

# Extract feature names from the preprocessor
feature_names = []
if hasattr(preprocessor_100, "transformers_"):
    for name, transformer, columns in preprocessor_100.transformers_:
        if name == "cat" and hasattr(transformer, "named_steps"):
            # Get OneHotEncoder feature names for categorical columns
            encoder = transformer.named_steps['onehot']
            feature_names.extend(encoder.get_feature_names_out(categorical_features_100))
        elif name == "num":
            # Add numeric feature names directly
            feature_names.extend(columns)

# Get the boolean mask of selected features
selected_mask = select_from_model.get_support()

# Ensure feature_names and selected_mask have the same length for alignment
# This is where the correction is made
feature_names = feature_names[1:]

# Filter feature names using the boolean mask, but now with aligned lengths
selected_features = [feature_names[i] for i in range(len(feature_names)) if selected_mask[i]]

# Print the results
print(f"Number of features before feature selection: {X_100_preprocessed.shape[1]}")
print(f"Number of features after feature selection: {X_100_selected.shape[1]}")
print("Selected features:")
print(selected_features)

Number of features before feature selection: 217
Number of features after feature selection: 23
Selected features:
['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'BsmtFullBath', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'YrSold', 'LandContour_Lvl', 'ExterQual_TA', 'BsmtQual_Ex', 'BsmtQual_Gd', 'BsmtExposure_Gd', 'GarageFinish_Unf']


In [ ]:
# # Feature selection using SequentialFeatureSelector
# sequential_selector = SequentialFeatureSelector(selector_model, n_features_to_select=10, direction='forward').fit(X_100_preprocessed, y_100)
# X_100_sequential = sequential_selector.transform(X_100_preprocessed)

In [28]:
# Feature selection using GeneticSelectionCV
genetic_selector = GeneticSelectionCV(selector_model, cv=3, verbose=1, scoring="neg_mean_squared_error",
                                      n_population=30, n_generations=15, n_gen_no_change=5,
                                      crossover_proba=0.5, mutation_proba=0.2, n_jobs=-1)
genetic_selector.fit(X_100_preprocessed, y_100)
X_100_genetic = genetic_selector.transform(X_100_preprocessed)

# Print the number of features before and after feature selection
print(f"Number of features before feature selection: {X_100_preprocessed.shape[1]}")
print(f"Number of features after feature selection: {X_100_genetic.shape[1]}")

Selecting features with genetic algorithm.
gen	nevals	avg                                        	std                                        	min                                        	max                                        
0  	30    	[-1.548088e+09  1.177333e+02  6.382145e+08]	[ 9.099850e+08  5.637135e+01  2.031418e+08]	[-5.890095e+09  7.000000e+00  3.187650e+08]	[-1.072906e+09  2.140000e+02  1.324380e+09]
1  	13    	[-1.198325e+09  1.537000e+02  5.553118e+08]	[ 1.056010e+08  3.147819e+01  1.159338e+08]	[-1.463836e+09  9.100000e+01  4.235049e+08]	[-1.072906e+09  2.100000e+02  9.309154e+08]
2  	25    	[-1.116979e+09  1.617667e+02  4.740337e+08]	[ 36924480.880497        25.849156  54810184.562676]	[-1.191796e+09  1.110000e+02  3.978524e+08]	[-1.037713e+09  2.090000e+02  6.263155e+08]
3  	16    	[-1.100868e+09  1.584000e+02  4.516102e+08]	[ 48195365.339338        20.54199   54341164.329457]	[-1.252261e+09  1.110000e+02  3.910401e+08]	[-1.034098e+09  2.090000e+02  5.766080e+08]
4  	